a) Identificati independentele din retea.

O si L sunt independete conditionat de S => când cunoaștem dacă emailul este spam sau nu (S), probabilitatea de apariție a cuvântului "ofertă" (O) și prezența linkurilor (L) sunt independente. => 𝑂⊥𝐿 ∣ 𝑆.

M depinde atat de S, cat si de L =>
Nu există independență condiționată între M, S si L deoarece ambele variabile influențează direct M.


In [ ]:
pip install pgmpy

In [5]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([
    ('S', 'O'),
    ('S', 'L'),
    ('S', 'M'),
    ('L', 'M')
])




cpd_S = TabularCPD(variable='S', variable_card=2, values=[[0.6], [0.4]])

cpd_O = TabularCPD(variable='O', variable_card=2,
                   values=[[0.9, 0.3], [0.1, 0.7]],
                   evidence=['S'], evidence_card=[2])

cpd_L = TabularCPD(variable='L', variable_card=2,
                   values=[[0.7, 0.2], [0.3, 0.8]],
                   evidence=['S'], evidence_card=[2])

cpd_M = TabularCPD(variable='M', variable_card=2,
                   values=[[0.8, 0.4, 0.5, 0.1], [0.2, 0.6, 0.5, 0.9]],
                   evidence=['S', 'L'], evidence_card=[2, 2])



model.add_cpds(cpd_S, cpd_O, cpd_L, cpd_M)

model.check_model()

active_trail_O_L_given_S = model.active_trail_nodes('O', observed=['S'])

independence_test = 'L' not in active_trail_O_L_given_S

independence_test


True

b) Determinati cum clasifica reteaua bayesiana emailuirle in functie de O, L si M.

In [6]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([
    ('S', 'O'),
    ('S', 'L'),
    ('S', 'M'),
    ('L', 'M')
])



cpd_S = TabularCPD(variable='S', variable_card=2, values=[[0.6], [0.4]])

cpd_O = TabularCPD(variable='O', variable_card=2,
                   values=[[0.9, 0.3], [0.1, 0.7]],
                   evidence=['S'], evidence_card=[2])

cpd_L = TabularCPD(variable='L', variable_card=2,
                   values=[[0.7, 0.2], [0.3, 0.8]],
                   evidence=['S'], evidence_card=[2])

cpd_M = TabularCPD(variable='M', variable_card=2,
                   values=[[0.8, 0.4, 0.5, 0.1], [0.2, 0.6, 0.5, 0.9]],
                   evidence=['S', 'L'], evidence_card=[2, 2])

model.add_cpds(cpd_S, cpd_O, cpd_L, cpd_M)

model.check_model()

inference = VariableElimination(model)



def classify_email(O_value, L_value, M_value):
    # Calculeaza P(S=1 | O, L, M)
    result = inference.query(variables=['S'], evidence={'O': O_value, 'L': L_value, 'M': M_value})
    prob_spam = result.values[1]  # P(S=1)
    prob_non_spam = result.values[0]  # P(S=0)

    classification = "Spam" if prob_spam > prob_non_spam else "Non-Spam"

    return prob_spam, prob_non_spam, classification


test_cases = [
    {'O': 1, 'L': 1, 'M': 1},
    {'O': 1, 'L': 0, 'M': 0},
    {'O': 0, 'L': 1, 'M': 1},
    {'O': 0, 'L': 0, 'M': 0}
]

for case in test_cases:
    prob_spam, prob_non_spam, classification = classify_email(case['O'], case['L'], case['M'])
    print(f"O={case['O']}, L={case['L']}, M={case['M']} -> P(Spam)={prob_spam:.4f}, P(Non-Spam)={prob_non_spam:.4f}, Classified as: {classification}")


O=1, L=1, M=1 -> P(Spam)=0.9492, P(Non-Spam)=0.0508, Classified as: Spam
O=1, L=0, M=0 -> P(Spam)=0.4545, P(Non-Spam)=0.5455, Classified as: Non-Spam
O=0, L=1, M=1 -> P(Spam)=0.4706, P(Non-Spam)=0.5294, Classified as: Non-Spam
O=0, L=0, M=0 -> P(Spam)=0.0382, P(Non-Spam)=0.9618, Classified as: Non-Spam
